In [ ]:
#Reset Button. Press when memory goes crazy fat !
%reset

In [1]:
# imports cell
import numpy as np
import _pickle as cPickle
import sys 
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Deep Learning libs
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten

Using TensorFlow backend.


# Let's start with data

We use DeepSig Dataset: RadioML 2016.04C<br>
A synthetic dataset, generated with GNU Radio, consisting of 11 modulations. This is a
variable-SNR dataset with moderate LO drift, light fading, and numerous different
labeled SNR increments for use in measuring performance across different signal and
noise power scenarios.

## Loading data

**The file is formatted as a "pickle" file which can be open for example in python by using cPickle.load(...)**



In [2]:
filename = "RML2016.10b.dat"
open_file = open(filename,'rb')
data = cPickle.load(open_file, encoding='latin1')

In [4]:
# Use lists when accessing data from dict for ease of access.
keys_list = list(data.keys())
temp_list = []
label_list = []
snr_list = []
for i in range(len(keys_list)):
    curr_item = data[keys_list[i]] 
    temp_list.append(curr_item)
    for j in range(curr_item.shape[0]):
        label_list.append(keys_list[i][0])
    #for j in rane(curr_item.shape[0]):
    #    snr_list.append(keys_list[i][1])

# Convert all lists into numpy arrays.
X = np.array(temp_list).reshape(1200000,2,128)
Y = np.array(label_list)

# Clear All un-neccsarry lists created.
temp_list.clear()
label_list.clear()
snr_list.clear()
keys_list.clear()
data.clear()

## Create feature Spaces for data

**Every sample is presented using two vectors each of them has 128 elements. Those two vectors are the in-phase and quadrature phase components of a sample.**

In [9]:
derivative_X = np.arange(1200000*4*128, dtype="float32").reshape(1200000,4,128)
derivative_X = np.concatenate((np.apply_along_axis(lambda column:np.gradient(column),2,X),X),axis=1)

In [5]:
integral_X = np.arange(1200000*4*128, dtype="float32").reshape(1200000,4,128)
integral_X =np.concatenate((np.apply_along_axis(lambda column:np.hstack((0,column)),2,np.apply_along_axis(lambda column:sp.integrate.cumtrapz(column,dx=1,axis=-1),2,X)),X),axis=1)

## Splitting data

**Split the data into 50% for training/validation and 50% for testing.**

In [12]:
(trainX, testX, trainY, testY) = train_test_split(X,Y, test_size=0.50, random_state=42)

## Encoding Labels

Our class labels are currently represented as strings; however, Keras will assume that both:

1. Labels are encoded as integers

2. And furthermore, one-hot encoding is performed on these labels making each label represented as a vector rather than an integer

To accomplish this encoding, we can use the LabelBinarizer  class from scikit-learn

In [13]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

# Defining the Fully Connected Model
<ul>

1. The core data structure of Keras is a model. The simplest type of model is the Sequential model, a linear stack of layers.

2. A 3-layer deep neural network consisiting only of fully connected layers of size 512, 256, 11 neurons.


In [25]:
# Declare layers size
in_shape = (2,128)
hidden1_size = 512
hidden2_size = 256
hidden3_size = 10

# Build model
model = Sequential()
model.add(Dense(hidden1_size, input_shape = in_shape))
model.add(Activation('relu'))
model.add(Dense(hidden2_size))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(hidden3_size))
model.add(Activation('relu'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 2, 512)            66048     
_________________________________________________________________
activation_30 (Activation)   (None, 2, 512)            0         
_________________________________________________________________
dense_34 (Dense)             (None, 2, 256)            131328    
_________________________________________________________________
activation_31 (Activation)   (None, 2, 256)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                5130      
_________________________________________________________________
activation_32 (Activation)   (None, 10)                0         
Total para

## Fitting the model

1. The training process will run for a fixed number of iterations (epochs) through the dataset.

2. We can also set the number of instances that are evaluated before a weight update in the network is performed (batch size).


In [ ]:
model.fit(trainX, trainY, epochs=150, batch_size=300000)

Epoch 1/150
600000/600000 [==============================] - 16s 27us/step - loss: 8.7334 - acc: 0.1752
Epoch 2/150
600000/600000 [==============================] - 15s 25us/step - loss: 8.7333 - acc: 0.1753
Epoch 3/150
600000/600000 [==============================] - 16s 26us/step - loss: 8.7332 - acc: 0.1755
Epoch 4/150
600000/600000 [==============================] - 15s 25us/step - loss: 8.7332 - acc: 0.1757
Epoch 5/150
600000/600000 [==============================] - 16s 26us/step - loss: 8.7333 - acc: 0.1759
Epoch 6/150
600000/600000 [==============================] - 15s 25us/step - loss: 8.7333 - acc: 0.1759
Epoch 7/150
600000/600000 [==============================] - 15s 25us/step - loss: 8.7333 - acc: 0.1759
Epoch 8/150
300000/600000 [==============>...............] - ETA: 7s - loss: 8.7391 - acc: 0.1755

In [17]:
a1 = np.array([[1,2,3],[4,5,6]])
azeros = np.zeros((2,1))

In [21]:
np.hstack((0,a1[0]))

array([0, 1, 2, 3])